<a href="https://colab.research.google.com/github/prateekrao/Breast-Cancer-Detection/blob/main/New_Densenet_train-false-86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/split-data.zip"

In [28]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D
from keras.applications import ResNet50, DenseNet169, DenseNet121, DenseNet201, MobileNetV2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2, l1_l2
import matplotlib.pyplot as plt

In [8]:
tgen = ImageDataGenerator(
    rescale = 1/255,
    samplewise_center = True,
    samplewise_std_normalization = True
)

In [9]:
train_set = tgen.flow_from_directory(
    "/content/split-data/train",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = True,
    seed = 865,
    color_mode = 'rgb'
)

Found 194266 images belonging to 2 classes.


In [10]:
ogen = ImageDataGenerator(
    rescale = 1/255
)

In [44]:
test_set = tgen.flow_from_directory(
    "/content/split-data/test",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = False,
    color_mode = 'rgb'
)

Found 41629 images belonging to 2 classes.


In [45]:
valid_set = tgen.flow_from_directory(
    "/content/split-data/test",
    target_size = (128,128),
    class_mode = 'sparse',
    shuffle = False,
    color_mode = 'rgb'
)

Found 41629 images belonging to 2 classes.


In [46]:
densenet = DenseNet121(include_top = False, weights = 'imagenet', input_shape = (128,128,3))


In [47]:
for layer in densenet.layers:
  layer.trainable = False

In [48]:
model = tf.keras.Sequential()

model.add(densenet)

model.add(Conv2D(64, activation='relu', kernel_size=(3,3)))
model.add(GlobalAveragePooling2D())

model.add(Dense(512, activation='relu', kernel_regularizer=l1(0.000001)))
model.add(Dropout(0.5))
model.add(Dense(512, activation= 'relu', kernel_regularizer=l2(0.0000001)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(0.00001)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(0.000001)))
model.add(Dense(2, activation='softmax'))

In [49]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 4, 4, 1024)        7037504   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 64)          589888    
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               33280     
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)              

In [50]:
opt = Adam(lr = 9e-05)

model.compile(optimizer=opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [51]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("/content/drive/MyDrive/densenet_and_conv_best.h5")
checkpoint

In [ ]:
history = model.fit(train_set, validation_data=valid_set, batch_size = 128, epochs = 20, shuffle = True)

Epoch 1/20
6071/6071 [==============================] - 734s 120ms/step - loss: 2.7779 - accuracy: 0.8349 - val_loss: 0.3277 - val_accuracy: 0.8657
Epoch 2/20
6071/6071 [==============================] - 724s 119ms/step - loss: 0.3254 - accuracy: 0.8680 - val_loss: 0.3235 - val_accuracy: 0.8678
Epoch 3/20
6071/6071 [==============================] - 724s 119ms/step - loss: 0.3118 - accuracy: 0.8720 - val_loss: 0.3177 - val_accuracy: 0.8699
Epoch 4/20
6071/6071 [==============================] - 724s 119ms/step - loss: 0.3064 - accuracy: 0.8744 - val_loss: 0.3152 - val_accuracy: 0.8683
Epoch 5/20
6071/6071 [==============================] - 715s 118ms/step - loss: 0.3015 - accuracy: 0.8772 - val_loss: 0.3154 - val_accuracy: 0.8682
Epoch 6/20
6071/6071 [==============================] - 719s 118ms/step - loss: 0.2945 - accuracy: 0.8796 - val_loss: 0.3163 - val_accuracy: 0.8705
Epoch 7/20
6071/6071 [==============================] - 717s 118ms/step - loss: 0.2883 - accuracy: 0.8822 - val_

# **TEST SET: DISABLE SAMPLEWISE_DIST AND EVALUATE**

In [ ]:
gennew = ImageDataGenerator(rescale = 1/255)

In [ ]:
test_set = gennew.flow_from_directory(
    "/content/split-data/test",
    target_size = (128,128),
    shuffle = False,
    color_mode = 'rgb',
    class_mode = 'sparse',
)

In [ ]:
model.evaluate(test_set)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy Graph")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot("Loss Graph")